In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, Dense, Flatten, Activation

import time
from sklearn.metrics import r2_score
import kerastuner as kt

from sklearn.model_selection import train_test_split

In [25]:
def build_model(hp):  # random search passes this hyperparameter() object
    model = keras.models.Sequential()
    model.add(Dense(hp.Int('input_units', min_value=4, max_value=12, step=2),
                     activation='relu', input_dim = X.shape[1]))
    model.add(Dropout(0.3))

    for i in range(hp.Int('n_layers', 0, 2)):  # adding variation of layers.
        model.add(Dense(hp.Int('input_units', min_value=4, max_value=12, step=2),
                     activation='relu'))

    model.add(Dense(6))

    model.compile(optimizer=keras.optimizers.Adam(lr=0.005),
                  loss="mean_absolute_error",
                  metrics=["mean_absolute_error"])
    return model

In [26]:
Dataset1_Smiles_P = pd.read_csv("dataset1_imputed_all.csv")
Dataset1_Smiles_P = Dataset1_Smiles_P.groupby('Smiles').mean().reset_index()
Dataset1_Smiles_P

,Smiles,Unnamed: 0,index,Year,He,H2,O2,N2,CO2,CH4,...,log10_O2_Bayesian,log10_N2_Bayesian,log10_CO2_Bayesian,log10_CH4_Bayesian,log10_He_Etree,log10_H2_Etree,log10_O2_Etree,log10_N2_Etree,log10_CO2_Etree,log10_CH4_Etree
0,*/C(=C(\*)c1ccc(-c2ccc(C(C)(C)C)cc2)cc1)c1ccccc1,750.5,269.5,2010.0,215.0,455.0,195.00,70.500,1220.00,190.000,...,2.264458,1.820241,3.064200,2.247077,2.322711,2.640971,2.264458,1.820241,3.064200,2.247077
1,*/C(=C(\*)c1ccc(-c2ccc([Si](C)(C)C)cc2)cc1)c1c...,743.0,262.0,2010.0,370.0,810.0,390.00,160.000,2200.00,450.000,...,2.591065,2.204120,3.342423,2.653213,2.568202,2.908485,2.591065,2.204120,3.342423,2.653213
2,*/C(=C(\*)c1ccc(-c2ccccc2)cc1)c1ccccc1,749.0,268.0,2010.0,200.0,440.0,180.00,65.000,1300.00,170.000,...,2.255273,1.812913,3.113943,2.230449,2.301030,2.643453,2.255273,1.812913,3.113943,2.230449
3,*/C(=C(\*)c1ccc(C(C)(C)C)cc1)c1ccc(O)cc1,707.0,226.0,2006.0,190.0,340.0,82.00,23.000,530.00,46.000,...,1.913814,1.361728,2.724276,1.662758,2.278754,2.531479,1.913814,1.361728,2.724276,1.662758
4,*/C(=C(\*)c1ccc(C(C)(C)C)cc1)c1ccc(O[Si](C)(C)...,703.0,222.0,2006.0,530.0,1000.0,470.00,210.000,2300.00,540.000,...,2.672098,2.322219,3.361728,2.732394,2.724276,3.000000,2.672098,2.322219,3.361728,2.732394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,*c1ccc2c(c1)C1(CC2(C)C)CC(C)(C)c2ccc(N3C(=O)c4...,653.0,144.0,2013.0,332.0,624.0,208.00,65.000,1523.00,129.000,...,2.318063,1.812913,3.182700,2.110590,2.521138,2.795185,2.318063,1.812913,3.182700,2.110590
349,*c1cccc(N2C(=O)c3ccc(Oc4ccc(C(C)(C)c5ccc(Oc6cc...,409.5,409.5,1994.0,9.4,NaN,0.37,0.051,1.33,0.036,...,-0.434351,-1.231009,0.121595,-1.291081,0.927353,0.736209,-0.434351,-1.295008,0.103035,-1.443242
350,*c1cccc(N2C(=O)c3ccc(Oc4ccc(C5(c6ccc(Oc7ccc8c(...,543.0,543.0,1989.0,NaN,NaN,0.30,NaN,NaN,NaN,...,-0.522879,-1.214076,0.077487,-1.185679,0.874861,0.748087,-0.522879,-1.311811,0.037426,-1.428914
351,*c1cccc(N2C(=O)c3ccc(Oc4ccc5c(c4)C4(CC(C)(C)c6...,542.0,542.0,1989.0,NaN,NaN,2.82,NaN,NaN,NaN,...,0.450249,-0.168164,1.061415,-0.075697,1.294916,1.406761,0.450249,-0.120979,1.151142,-0.084523


In [27]:
X_fingerprints = pd.read_csv('X_fingerprints.csv')
X_fingerprints = X_fingerprints.drop(X_fingerprints.columns[0], axis=1)
X_fingerprints

,298,380,423,619,724,799,866,984,1116,1160,...,2706,2728,2814,2818,2854,2857,2925,2984,3093,3155
0,0,0,0,1,2,0,0,0,0,2,...,0,0,0,0,3,0,2,2,0,13
1,0,0,0,0,2,0,0,0,0,2,...,0,0,0,0,3,0,2,2,0,13
2,0,0,0,0,2,0,0,0,0,2,...,0,0,0,0,0,0,4,2,0,14
3,0,0,0,1,4,0,0,0,0,2,...,0,0,0,0,3,0,0,2,0,8
4,1,0,0,1,4,0,0,0,0,2,...,0,2,0,0,8,0,0,2,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,0,6,0,0,0,0,0,4,1,0,...,4,0,4,2,8,6,0,0,0,10
349,2,3,0,2,0,0,0,4,1,0,...,4,6,0,2,2,0,0,0,0,18
350,2,5,0,0,0,0,0,4,1,0,...,4,2,5,2,2,1,0,0,2,16
351,2,5,0,0,0,0,0,4,1,0,...,4,4,2,2,4,3,0,0,0,16


In [28]:
X = X_fingerprints
Y = Dataset1_Smiles_P.iloc[:,-12:-6]
Y = np.array(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [39]:
tuner = RandomSearch(
    build_model,
    #objective='mean_absolute_error',
    objective=kt.Objective('val_mean_absolute_error', 'min'),
    max_trials=100,  # how many model variations to test?
    executions_per_trial=1,  # how many trials per variation? (same model could perform differently)
    directory='tuner_logs')


tuner.search(x=X_train,
             y=Y_train,
             verbose=2, # just slapping this here bc jupyter notebook. The console out was getting messy.
             epochs=40,
             batch_size=64,
             #callbacks=[tensorboard],  # if you have callbacks like tensorboard, they go here.
             validation_data=(X_test, Y_test))

Trial 12 Complete [00h 00m 03s]
val_mean_absolute_error: 0.8206368088722229

Best val_mean_absolute_error So Far: 0.7268052101135254
Total elapsed time: 00h 00m 38s
INFO:tensorflow:Oracle triggered exit
